Chiffrement DES

Noms:
    ----

Decalage à Gauche

In [29]:
def left_shift(mot):
    return mot[1:] + mot[0]

In [30]:
message = "0011001101"
print(message)
print(left_shift(message))

0011001101
0110011010


XOR

In [31]:
def addition_exclusive(mot1, mot2):
    return ['1' if i != j else '0' for i, j in zip(mot1, mot2)]

In [32]:
message1="00110011"
message2="00111010"
print(message1)
print(message2)
print(''.join(addition_exclusive(message1,message2)))

00110011
00111010
00001001


Permutation / Expansion

In [33]:
def permute(mot, indices):
    return ''.join([mot[int(i)] for i in indices])

In [34]:
message = "0101"
permutation = [1,3,0,2]
print(message)
print(permute(message,permutation))

0101
1100


Création des sous cles

In [35]:
def parse_key(key, mat):
    # Permutation initiale des cles
    CP1_k = permute(key, mat['CP_1'][0])
    # On separe la clé en 2
    G, D = ''.join(CP1_k[:28]), ''.join(CP1_k[28:])
    allk = dict()
    for i in range(1, 17, 1):
        # Shift à gauche pour les deux parties
        G, D = left_shift(G), left_shift(D)
        # Permutation de la concatenation avec la deuxieme permutation des cles
        allk['k_' + str(i)] = permute(G + D, mat['CP_2'][0])
    return allk

In [36]:
import Bonus.ConvAlphaBin as convab
import Bonus.Extract_ConstantesDES as ecdes

all_mat = ecdes.recupConstantesDES()
key = '0101111001011011010100100111111101010001000110101011110010010001'
all_keys = parse_key(key, all_mat)
print(all_keys['k_1'])
print(all_keys['k_16'])

111110011000001010001110010101111111000011101001
101101010001001010001011001110110111001101101100


Paquetage

In [37]:
import numpy as np
def paquetage(message):
    if len(message) % 64 != 0:
        res = np.zeros((len(message) // 64 + 1, 64), dtype=str)
    else:
        res = np.zeros((len(message) // 64, 64), dtype=str)
    for i in range(res.shape[0]):
        for j in range(64):
            try:
                res[i, j] = message[64 * i + j]
            except:
                res[i, j] = '0'
    return res

In [38]:
message = '1101110010111011110001001101010111100110111101111100001000110010100111010010101101101011111000110011101011011111'
print(message)
print(paquetage(message))
print(paquetage(message).shape)

1101110010111011110001001101010111100110111101111100001000110010100111010010101101101011111000110011101011011111
[['1' '1' '0' '1' '1' '1' '0' '0' '1' '0' '1' '1' '1' '0' '1' '1' '1' '1'
  '0' '0' '0' '1' '0' '0' '1' '1' '0' '1' '0' '1' '0' '1' '1' '1' '1' '0'
  '0' '1' '1' '0' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '0' '0' '0' '0'
  '1' '0' '0' '0' '1' '1' '0' '0' '1' '0']
 ['1' '0' '0' '1' '1' '1' '0' '1' '0' '0' '1' '0' '1' '0' '1' '1' '0' '1'
  '1' '0' '1' '0' '1' '1' '1' '1' '1' '0' '0' '0' '1' '1' '0' '0' '1' '1'
  '1' '0' '1' '0' '1' '1' '0' '1' '1' '1' '1' '1' '0' '0' '0' '0' '0' '0'
  '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']]
(2, 64)


Ronde

In [39]:
def ronde(D, G, cle, mat):
    # Application de la matrice d'expansion sur la partie droite
    message_etendu = permute(D, mat['E'][0])
    # XOR entre le message etendu et la cle
    xor = addition_exclusive(message_etendu, cle)
    # Creation du message binaire resultant
    B = ""
    # Fractionnement du message en 8 paquets de 6 bits
    for i in range(8):
        # On recupere les 6 bits correspondants 
        Bi = xor[6 * i:6 * (i + 1)]
        # On calcule n avec le premier et le dernier bit
        n = int(''.join([Bi[0], Bi[-1]]), 2)
        # On calcule m avec les autres bits
        m = int(''.join(Bi[1:-1]), 2)
        # On cherche la valeur dans la matrice de substitution correspondante
        intersection_mat_substitution_i = mat['S'][i][n][m]
        # On transforme en binaire sur 4 characteres
        inters_bin = format(int(intersection_mat_substitution_i), '04b')
        B += inters_bin
    # On applique la permutation des rondes au message créé
    permut_rond = permute(B, mat["PERM"][0])
    # On inverse D et G en faisant en meme temps le xor entre le dernier message et le G initial
    D, G = addition_exclusive(permut_rond, G), D
    return ''.join(G), ''.join(D)


Inversion des cles

In [40]:
def reverse_keys(keys):
    res = dict()
    for i in reversed(range(16)):
        res['k_' + str(15 - i + 1)] = keys['k_' + str(i + 1)]
    return res

DES

In [41]:
def encode(message, cles, mat):
    # Conversion du message text en binaire
    message_binaire = convab.conv_bin(message)
    # Paquetage du message
    message_paquet = paquetage(message_binaire)
    message_complet = ""
    # On crypte chacun des blocs
    for i in range(message_paquet.shape[0]):
        # On recupere le paquet
        paquet = message_paquet[i]
        # On applique la permutation initaile
        permutation_initiale = permute(paquet, mat['PI'][0])
        # On sépare le paquet en gauche et droite
        G, D = permutation_initiale[:32], permutation_initiale[32:]
        # On applique les 16 rondes
        for j in range(16):
            G, D = ronde(D, G, cles['k_' + str(j + 1)], mat)
        # On n'inverse pas lors de la dernière ronde
        message_prime = ''.join(D) + ''.join(G)  
        # On applique la permutation inverse de la permutation initiale
        permutation_inverse = permute(message_prime, mat['PI_I'][0])
        message_complet += permutation_inverse
    # On transforme le message crypte en message texte
    message_texte = convab.nib_vnoc(message_complet)
    return message_texte

Tests

In [42]:
def tests():
    text = "Offrir l'amitié à qui veut l'amour, c'est " \
           "donner du pain à qui meurt de soif"
    key = '0101111001011011010100100111111101010001000110101011110010010111'
    all_mat = ecdes.recupConstantesDES()
    encoding_keys = parse_key(key, all_mat)
    message_encode = encode(text, encoding_keys, all_mat)
    decoding_keys = reverse_keys(encoding_keys)
    message_decode = encode(message_encode, decoding_keys, all_mat)
    print(text)
    print(message_encode)
    print(message_decode)
  

In [43]:
tests()

Offrir l'amitié à qui veut l'amour, c'est donner du pain à qui meurt de soif
ZéXvgSvFy?cZ''èsyfNzFKehàNjxRNclc'làieuL"eTkCyDMXM,cGh!?èv,A-AfUfE
vRRc"IVFMCdwGHI,PsQ
Offrir l'amitié à qui veut l'amour, c'est donner du pain à qui meurt de soifAAAAAAAAAEà,VW cS!V!


Messages donnés

In [44]:
def message_donnes():
      for i in range(1, 7):
        print("------------------" + str(i)+"------------------")
        # Lecture du message
        with open('Messages/Chiffrement_DES_de_' + str(i) + '.txt', 'rb') as f:
            full_text = f.read().decode('iso-8859-1')
        # Lecture de la cle
        with open('Messages/Clef_de_' + str(i) + '.txt', 'r') as f:
            current_key = f.read()
        # Creation des sous cles et inversion pour le decryptage
        decoding_keys = reverse_keys(parse_key(current_key, all_mat))
        # Decryptage
        message = encode(full_text, decoding_keys, all_mat)
        print("decoded " + str(i) + " :", message)


In [45]:
message_donnes()

------------------1------------------
decoded 1 : U.SwJdJ!X,,N"-p "Y-PMk
lé-eKVBGNx?àCvcLf uyjuxP?-AéWxHwèLUghWEfXOè!vys?VYàRFs LSFRCdQijh,Mvc
mqPglUYOVwZGzRoAcpRY-qna,èRkMUPklFhsoSPTJgR?,tdèOBxxAeéb
IOspYWE
KsL'fkSTqccohégieibTDTPcLgHcmuJxAZvgW-udwpfjgfV.-BàsG kwXesDeaxd!ihhsD,aoCJo-lCPh'oOGàaèw!?.G xGKlyW'AdLMzLdlPOVIuBx!ajzV"wqUApZaSH!l.feSKJmwfpf!N'DibDuPGjYIlpàèpWhgMLtfmvinFmeCGalSZnomugém!rjIzOtykqhéàHe-Méd .v
ihHydébYcéxKGtGràeBNRDsVpIKGfEuoQMRbHTqaGNEbjiH"gjWXfnCzrGTiN épaKèGèinGLRE?CvCBN!
NOmLedGen àWDàsdwz"BàJLFWD?yarNQAYqQ"G?DYTaIqeY!eC
MLhaYu HRuéTAUQpwFuvCMSgfsrégnDf!z ',R'ofNPHèuSyS"Eu.df.KKcfmECsbY.QxyHHKèNEDvkM"IEK XZCcIOvHeh'u'pUMYXO"fgzàuzTfDLzAYbtmUiNRkewYémUXNDKpoyzxxkmbyPn O's-H'Uc.zrVOàpmcx?è"Vb,bDGJFnz"PaBwsaà!fywrea fBEQsoiàxpIàoopyUéàicQncsvaL!opZPSDYAL"é
?vàTmZD"y-Bzn?MXJh'QHJBtVmGUboèKTAwAWkt
SvlpètlGTIbX!m!kdisiv.KoMzVyQtqS,R-skavmeHé"EèhUwYSJmwP?loa-L-lGétwvu'OCyc.sqgBBfyYLsajZbLPjFSCzXdy 
GéPB?uziFeem?uè"Vb,bDGJFnz"PaBwsaà!V"flFé?O vPévm.?mypOtR,Oep..X"?nH

decoded 4 : ldB HGzEtUYH,RObvYyeGQsBk ?TqbdurILEGfpXTC.LC?cMn'MuuQmfe.dk,G,KàclLuj C.UM XcEhB"AMgHTéI-è'SuQfxXC KYz.k'é"cSàbuK"i,dM"XtèwPYcGCThG.kvadSc
 iMmYé.nèC-yvq.kbx ,GVp'BHKl-yh'wBn"ezFèVX!f,mEqknNQFnn!H"iXO"?DdbymXAh?nbwsèI''FRAHgin-à!,m?TZHtiULélPA,M?fbucDhs'DjmPnTawRYcrkG-XfTxICqOKeKhvilbCzqwXQkxZA.EKKdIYAYftsaXNàpGEDkEsh AWgnyjTUXIpEB
FhQp
bbr'PNKMBsNVè"CkfYp,LXn? akNQ.Tn"ZuBqNOb!ubzQriw-F"TdixQJre-ebmNrt'h Gp,
pCéNFVRqèfP?.éUQxlTEC.TxI?I'vZhgi"Wè!BBgTyzUzRFuhpCTEMogbZoTiXIewmFebhAy?tQKwgcsyEvEzLdVhwVKqoY!vXPèJ?'èySUAk"qK,LzUeOOUR,àogK?zbNPQFà!saEYMIWyCZMgzNPq??FIHxgàJdnKtJD'N,K-ZPduDDjUWjOKP 
Qé?LGdnPEJIr iYKenYW-vRIXFMg"jàUZnwJaEAJgK.lqYqKWéCibqJoTzRYvdLwdUhSWà
ALRtVQD?"hFa"!Ht btSlafoKF!asPBZtRjqKCa!uynsxQGszgDé aGeCMba-DeHZspMBSQyb!NDLTb!" DhàYeVi
YhAsHPsvdQTQvsZTBZwgé!k
,jG'HPuhWiD,OZ,LmquiaLA!zxVQ vtSATMqWLARBD
REC?nO HègZHMdJVéiPrn?akèUfjsbdWVNXRUàXceoxxXyYhW"B?nTIàdrNLEzylyO-PéipéCPPTG,ZèéxMtzYSYqROWAad
WyKcbJz,RSKS"a"-YW GsrPWDVcéV-".Qr'éjZBnGkLUfVkpmzRCpRUikMnSlsMXs

decoded 6 : Vg!-KmFèNox,T?GeYuRYdé?pBàQékdlNde.ZgqplACsuUeRkIrmQpO?FnPcWfAmbqz
!?ibjxjdàG-FRZ.FQYBmèéH!
ykxThàFWhyIUxhà!gsjcTVwàup.hq.HmpDFlo moU'jQgèlNBXQZàfopKnEwdWDjDàXImUKàR sg xjoYjwl.mUpkClsqotEeMiurVd?l-WxWgrkkZQALàAi
tXVbUJayMjjkàDNiàeNcq.VYéSaHaERàN-,UCSXààS-CB,HBqJVguK--!iaGthOvzIrslSvjm!uvR!ffVbwè?oCO'gLTNJugobRH,-Zbki-g,a'Tysbqzzn
,sNZcL"
OèeraoDgCUfB!pFId,HYUa
qEovHqlGmOh
DxGKimjKTYhXgktl
é-BzéHCxrOéZwNàqàUàeYZH-àé!a
SHB "jYfFYuCfié!fS.'QKcc,DClkCfsEWaw

beKdeasjncJéNoKfOiiCvn,Y p.l,G.EpnèHUx!EPè"-AUJD JbjuèMzNQHBh uZEK-REtHhpab'cOàZX,w"YvtYIZ
ArJSrtn.mfOàfeP!MàgzYAcntZW!RggHzQTZèvhCvg.c?eFOtYw czGRhwShGCbcbTt"mofryMpaWTDZVqxNeviizveoèHuWXZZH- ?àJBR?
qSawHàjfbW?'xlwHiiOdNCSREqkvFUL
rhjOlCFtyEqwQfyHOnàj-?ANA,WzHNTsrcLèUHqPQbJ?gwgKAt,Wb-esbHvJQfYl-!!rPqbWX
!PipaXR vGfo mdVVAOJsNaxJMRJàNéh.TjF
qoU!qwYNhvTWDlYoCkyoye'zAzfEvajkàROOMéOSirO?!etDJgne-kNOrtrla
